In [189]:
import numpy as np
import pandas as pd
import nltk
import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re

In [208]:
bing_liu=pd.read_csv("../input/nlp-assignnmet4/BingLiu.csv",names=["word","polarity"],delimiter="\t")
dictionary=open("../input/nlp-assignnmet4/english-hindi-dictionary.txt",'rb')

In [209]:
string=dictionary.readlines()

In [210]:
dictionary_list=[]
for i in string:
    dictionary_list.append(i.decode('utf-8').replace("\n","").split("|||"))

In [211]:
dic_eng_hindi={}
dic_hindi_eng={}
for i in dictionary_list:
    if(i[0].replace(" ","")!=i[1].replace(" ","")):
        if(i[0][:-1] in dic_eng_hindi):
            dic_eng_hindi[i[0][:-1]].append(i[1][1:])
        else:
            dic_eng_hindi[i[0][:-1]]=[i[1][1:]]
        if(i[1][1:] in dic_hindi_eng):
            dic_hindi_eng[i[1][1:]].append(i[0][:-1])
        else:
            dic_hindi_eng[i[1][1:]]=[i[0][:-1]]


In [223]:
file_english=open("../input/nlp-assignnmet4/english.txt",'r')
english=file_english.read()
# remove multiple spaces
english = re.sub(r'\s+',' ',english)
# lower the text
english = english.lower()
english = re.sub(r'\d',' ',english)
english = re.sub(r'\s+',' ',english)
english_sentences = nltk.sent_tokenize(english)
lis=[]
tokenizer = nltk.RegexpTokenizer(r"\w+")
for sentence in english_sentences:
    lis.append(tokenizer.tokenize(sentence))
english_sentences=lis
for i in range(len(english_sentences)):
    english_sentences[i] = [word for word in english_sentences[i] if (word not in stopwords.words('english'))]

# Training the Word2Vec model
model_english = gensim.models.Word2Vec(english_sentences,size=150 ,min_count=1,window=15)
w2v_english_words = model_english.wv.vocab

In [224]:
import string  
# Storing the sets of punctuation in variable result  
punctuataions = list(string.punctuation) 
punctuataions.append('।')

# Storing the sets of punctuation in variable result  
file_hindi=open("../input/nlp-assignnmet4/hindi.txt",'r')
hindi=file_hindi.read()
hindi = re.sub(r'\s+',' ',hindi)
hindi = re.sub(r'\d',' ',hindi)
hindi = re.sub(r'\s+',' ',hindi)
hindi_sentences = nltk.sent_tokenize(hindi)


lis=[]
for sentence in hindi_sentences:
    lis.append(nltk.word_tokenize(sentence))
hindi_sentences=lis
for i in range(len(hindi_sentences)):
    hindi_sentences[i] = [word for word in hindi_sentences[i] if word not in punctuataions]

model_hindi = Word2Vec(hindi_sentences,size=150 ,min_count=1,window=15)
w2v_hindi_words = model_hindi.wv.vocab



In [240]:
billingual_lexicon=[]
for i in range(len(bing_liu['word'])):
    if(bing_liu['word'][i] in dic_eng_hindi):
        temp_eng=bing_liu['word'][i]
        for j in dic_eng_hindi[temp_eng]:
            billingual_lexicon.append([temp_eng,j,bing_liu['polarity'][i]])

In [241]:
Old_List=billingual_lexicon
AdditionFlag=True
while(AdditionFlag):
#     print(Old_List)
#     print(len(Old_List))
    New_List=[]
    AdditionFlag=False
    for entry in Old_List:
#         print(entry)
        e_word=entry[0]
        h_word=entry[1]
        if(e_word in w2v_english_words):
            eng_5_close_words=model_english.wv.most_similar(e_word,topn=10)
        else:
            continue
        if(h_word in w2v_hindi_words):
            hin_5_close_words=model_hindi.wv.most_similar(h_word,topn=10)
        else:
            continue
        for i in eng_5_close_words:
            if(i[0] in dic_eng_hindi):
                for j in hin_5_close_words:

                    for k in dic_eng_hindi[i[0]]:

                        if(k==j[0]):
#                             print(e_word," : ",i[0],"  ",h_word," : ",j[0],k,entry[2])

                            temp_addition=[i[0],j[0],entry[2]]
                            temp_positive=[i[0],j[0],'positive']
                            temp_negative=[i[0],j[0],'negative']
#                             print(temp_addition)

                            if(temp_negative not in billingual_lexicon and temp_positive not in billingual_lexicon and temp_positive not in New_List and temp_negative not in New_List):

                                print(temp_addition)
                                New_List.append(temp_addition)
                                AdditionFlag=True
    billingual_lexicon.extend(New_List)
    Old_List=New_List

['usage', 'उपयोग', 'negative']


In [236]:
L1=billingual_lexicon

In [237]:
L1

[['abundant', 'हुस्न', 'positive'],
 ['accessible', 'सुलभ', 'positive'],
 ['accomplish', 'पूरा', 'positive'],
 ['accomplished', 'पूरा हुआ', 'positive'],
 ['accomplishment', 'उपलब्धि', 'positive'],
 ['accomplishments', 'उपलब्धियों', 'positive'],
 ['accurate', 'सटीक', 'positive'],
 ['accurately', 'यथासंभव', 'positive'],
 ['achievement', 'उपलब्धि', 'positive'],
 ['achievements', 'उपलब्धियाँ', 'positive'],
 ['acumen', 'कौशल', 'positive'],
 ['adjustable', 'समायोजनीय', 'positive'],
 ['admirable', 'प्रशंसनीय', 'positive'],
 ['admire', 'प्रशंसा', 'positive'],
 ['admirer', 'प्रशंसक', 'positive'],
 ['adorable', 'प्यारे', 'positive'],
 ['adore', 'प्यार करता हूँ', 'positive'],
 ['advanced', 'उन्नत', 'positive'],
 ['advantage', 'लाभ', 'positive'],
 ['advantages', 'लाभ', 'positive'],
 ['advocates', 'हिन्दुत्व की', 'positive'],
 ['affection', 'स्नेह', 'positive'],
 ['affinity', 'संबंध', 'positive'],
 ['affirmative', 'सकारात्मक', 'positive'],
 ['afford', 'बर्दाश्त', 'positive'],
 ['affordable', 'किफाय